# Process well plate

Make sure this notebook runs in a GPU runtime.
Go to Runtime->Change Runtime Type' and select GPU

# Ceck GPU 

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

# Load Dependencies

Will restart kernel upon completion

In [ ]:
import os
# Clone repository
!git clone https://github.com/e11bio/well-plate-assay.git
# install module
!pip install -e /content/well-plate-assay/.
# install dependencies.
!pip install zarr
!pip install datashader
!pip install nd2
!pip install napari
!pip install cellpose
!pip install torch torchvision torchaudio
!pip install panel
!pip install jinja2
!pip install flask
!pip install seaborn --upgrade
# restart kernel.
os.kill(os.getpid(), 9)

## Setup google drive connection
Make sure you have a 'Manager' permission for the required folder.

Will open additional windows to gain access.

In [ ]:
from google.colab import drive,auth
import gspread
from google.auth import default

# mount drive
drive.mount('/content/drive/')
# setup google sheets connectivity
auth.authenticate_user()
gc = gspread.authorize(default()[0])

## File Selection

In [ ]:
nd2_file = '/content/drive/Shareddrives/GDriveData/Ab Screen (HEK) - EPITOPE TAGS/220618_HEK002_ct1-ct3/220714_96w-Merged_DAPI-GFP-555-647.nd2'
sheet_file = "HEK002 (ct1 vs ct3) | Imaging REFs & Well IDs" # just the sheet name !
sheet_tab = 'HEK002_MAPS' # Worksheet name (Tab) that contains the Map.

## Read in data

In [ ]:
from wellplate.elements import read_plate_gsheet
from wellplate.elements import read_nd2
from wellplate.plots import plate_map

meta_data = read_plate_gsheet(gc, sheet_file, sheet_tab)
im_data, channel_names, colormaps = read_nd2(nd2_file)
display(f'Available channels in nd2 file: {channel_names}')
plate_map(meta_data)

# Get cell segmentation.
Will take anywhere from 20-30 mins.

In [ ]:
from wellplate.extract import run_cellpose

run_cellpose(nd2_file,'365 nm', redo=False)

# Get cell intensities

In [ ]:
from wellplate.extract import calculate_intensities_channel

calculate_intensities_channel(nd2_file, '365 nm', '488 nm', redo=False)
calculate_intensities_channel(nd2_file, '365 nm', '640 nm', redo=False)

## Calculate scaffold/epi ratio result

In [ ]:
from wellplate.extract import calculate_scaffold_epi_ratios

results = calculate_scaffold_epi_ratios(nd2_file, meta_data, '488 nm', '640 nm', threshold_factor = 0.5)
display(results)

# Examples

## Grouping

In [ ]:
display(results.groupby(['SCAFFOLD','EPITOPE','PRIMARY-01']).mean())

## Compare Scaffold

In [ ]:
import seaborn as sns

g = sns.catplot(
    data=results, kind="bar",
    x="EPITOPE", y="ratio", hue="SCAFFOLD", errorbar="se", palette="dark", alpha=.6, height=6
)
g.set_axis_labels("", "Ratio Scaffold/Epitope")
g.legend.set_title("")

## Compare Primaries per Epitope


In [ ]:
import seaborn as sns
for epi in results['EPITOPE'].unique():
  g = sns.catplot(
      data=results.loc[results.EPITOPE==epi], kind="bar",
      x="EPITOPE", y="ratio", hue="PRIMARY-01", errorbar="se", palette="dark", alpha=.6, height=6
  )
  g.set_axis_labels("", "Ratio Scaffold/Epitope")
  g.legend.set_title("")

# Appendix

## Show significant cell masks

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from wellplate.plots import show_sig_cell_masks

show_sig_cell_masks(nd2_file, well_ind= 10, dapi_channel='365 nm', scaffold_channel='488 nm', threshold_factor = 0.5)